In [10]:
import os, sys
import traceback
import pandas as pd
import re
import glob
import time
from string import ascii_uppercase
from natsort import natsorted
import math


print('\t',os.getcwd(), '\n')

def write_to_subfolder(result_df, new_file, subdirectory):
    
    curr_dir = os.getcwd()
    new_dir = os.path.join(curr_dir,subdirectory)
    counter = 0
    
    if curr_dir.find(subdirectory) == -1:
        
        try:
            os.mkdir(subdirectory)
        except Exception:
            pass
    
        os.chdir(new_dir)
        print('File written  :-  ', new_file)
        result_df.to_csv(new_file, index = False)
        counter = 1
        os.chdir('..')
        
    else:
        os.chdir(new_dir)
        print('File written  :-  ', new_file)
        result_df.to_csv(new_file, index = True)
        os.chdir('..')
        

def Create_Output():

    df_all = pd.DataFrame()
    df_temp = pd.DataFrame()

    filenames = glob.glob("bvmo_primers*.csv")
    filenames = natsorted(filenames)
    Lengths = []

    for file in filenames:
        print("File read:-  ", file)
        df_temp = pd.read_csv(file)
        
        write_384well_IDT_files(df_temp, 'bvmo_primers_')
        
        Lengths.append(len(df_temp))
    
    print(" \n \t Total number of primers:-  ", sum(Lengths) )
                    
            
    
def generate_IDT_384well_format(df):
    
    df  = df.reset_index(drop=True)
    df_384well_template = well_plate_384()
    column_names = ["Well Position", "Name", "Sequence"]
    df_temp = pd.DataFrame(columns = column_names)
    total_DNA = len(df['Sequence'])
    print('\n \t',total_DNA, " <<-- This value should be less than 384 \n")

    for index, row in df_384well_template.iterrows():
        Name = df['Name'].loc[index]
        Sequence = df['Sequence'].loc[index]
        position = df_384well_template['Well Position'].loc[index]
        list1 = [position, Name, Sequence]
        a_series = pd.Series(list1, index = df_temp.columns)
        df_temp = df_temp.append(a_series, ignore_index=True)

        if index+1 == total_DNA:
            break
   
    return df_temp


            
def write_384well_IDT_files (df_input, name):
        
    Plates = math.ceil(len(df_input['Name'])/384)

    for number in range(Plates):
        df_IDT = pd.DataFrame()
        df_pass = pd.DataFrame()
        start_ind = 384 * (number)
        end_ind = 384 * (number +1)

        if Plates == 1:
            df_IDT = generate_IDT_384well_format(df_input)
        else:
            df_pass = (df_input[start_ind:end_ind])
            df_IDT = generate_IDT_384well_format(df_pass)

    IDT_filename = name + '384wellIDT_' + str(number+1)+'.csv'
        
    write_to_subfolder(df_IDT, IDT_filename, 'IDT_Ordering')

             
def well_plate_384():

    column_names = ["Well Position",  "Name", "Sequence"]
    df_384well_template = pd.DataFrame(columns = column_names)
    counter = 0

    for letter in ascii_uppercase:

        for i in range(24):
            counter = counter + 1
            Name = letter+str(i+1)

            list1 = [Name, '', '']
            a_series = pd.Series(list1, index = df_384well_template.columns)
            df_384well_template = df_384well_template.append(a_series, ignore_index=True)      

        if letter == 'P':
            break
    
    return df_384well_template
    

    
if __name__ == '__main__':
    
    start_time = time.time()

    Create_Output()
    runtime = (time.time() - start_time)
    print("\nRuntime - - - {:.2f} seconds  - - -".format(runtime))

	 /Users/nilmani/Desktop/Protein_Engineering 

File read:-   bvmo_primers.csv

 	 192  <<-- This value should be less than 384 

File written  :-   bvmo_primers_384wellIDT_1.csv
 
 	 Total number of primers:-   192

Runtime - - - 0.36 seconds  - - -
